## Import Libraries

In [2]:
import os
import sys
import time
import struct
import numpy as np
import matplotlib.pyplot as plt

path = os.path.abspath('../../')
if not path in sys.path:
    sys.path.append(path)
import mmwsdr

## Parameters

In [3]:
nread = 1024  # num of continuous samples per batch
nskip = 1024  # num of samples to skip between batches
nbatch = 1  # num of batches
tx_pwr = 10000  # transmit power
isdebug = True

## Create an SDR object

In [4]:
sdr0 = mmwsdr.sdr.Sivers60GHz(ip='10.113.6.3', unit_name='SN0243', isdebug=isdebug)
sdr0.fpga.configure('../../config/rfsoc.cfg')

# Make sure that the nodes are not transmitting
sdr0.send(np.zeros((nread,), dtype='int16'))

timeout: timed out

## Transmit data

In [ ]:
# Create a signal in frequency domain
sc = 100  # subcarrier index.
nfft = nread
txfd = np.zeros((nfft,), dtype='int16')
txfd[(nfft >> 1) + sc] = 1
txfd = np.fft.fftshift(txfd, axes=0)

# Then, convert it back to time domain
txtd = np.fft.ifft(txfd, axis=0)

# Set the tx power
txtd = txtd / np.mean(np.abs(txfd)) * tx_pwr
sdr0.send(txtd)

## Receive data

In [ ]:
nbatch = 10

rxtd = sdr0.recv(nread, nskip, nbatch)

rxfd = np.fft.fft(rxtd, axis=1)
rxfd = np.fft.fftshift(rxfd, axes=1)
f = np.linspace(-nfft / 2, nfft / 2)
for ibatch in range(nbatch):
    plt.plot((abs(rxfd[ibatch, :])), '-')
plt.show()

In [ ]:
np.mean(rxtd)
# nsamp = nbatch * nread * sdr0.fpga.nch * 2
# sdr0.sock(b'+ %d %d %d' % (nread/sdr0.fpga.npar, nskip / sdr0.fpga.npar, nsamp * 2))